In [32]:
pump_address = '01'
length_pdu = '08'
command_characters = '57 4C'
flow_rate = 40000000 # nL/min ()
state1 = '01' # 01 for start pump, 00 for stop pump, 11 for prime pump
state2 = '00' # 00 for counter-clock wise, 01 for clock wise

# The customXOR function is used to strip whitespace characters
# from the PDU and split it by any whitespace in order to compute
# the XOR of the different bytes within the PDU

def customXOR(input_string):
    # strip whitespace characters and split by any whitespace
    hex_values = input_string.strip().split()

    # compute the XOR of the different bytes within the PDU
    result = 0
    for value in hex_values:
        result ^= int(value, 16)

    # return the XOR result of the PDU
    return hex(result)[2:]

def generate_pdu(command_characters,flow_rate,state1,state2):
    # make sure the flow rate is interger
    flow_rate_int = int(flow_rate)
    # convert to hex number
    hex_flow_rate = hex(flow_rate_int)
    # remove the prefix 0x and make 8 digit str
    hex_flow_rate_str = hex_flow_rate[2:].zfill(8)
    # split each 2 digit
    chunks = [hex_flow_rate_str[i:i+2] for i in range(0, len(hex_flow_rate_str), 2)]
    # combine with ''
    formatted_string = ' '.join(chunks)
    # combine all parameters
    all_chunks = [command_characters,chunks[0],chunks[1],chunks[2],chunks[3],state1,state2]
    formatted_string = ' '.join(all_chunks)
    return formatted_string

def generate_fcs(pump_address,length_pdu,pdu):
    xor_pdu = customXOR(pdu)
    # calculate the frame check sequence (hex input and output base)
    frame_check_sequence = "{:x}".format(int(pump_address, 16) ^
    int(length_pdu, 16) ^ int(xor_pdu, 16))
    return frame_check_sequence

# https://blog.darwin-microfluidics.com/control-command-string-generator-for-longer-peristaltic-pumps/
# Set running parameter (flow rate)
def generate_pump_command(flow_rate,state1,state2):
    start_flag = 'E9'
    pdu = generate_pdu(command_characters,flow_rate,state1,state2)
    frame_check_sequence = generate_fcs(pump_address,length_pdu,pdu)

    params = [start_flag,pump_address,length_pdu,pdu,frame_check_sequence]
    hex_values = ' '.join(params).split()
    result = bytearray(int(value, 16) for value in hex_values)

    return hex_values, result

generate_pump_command(flow_rate,state1,state2)
    

(['E9', '01', '08', '57', '4C', '02', '62', '5a', '00', '01', '00', '29'],
 bytearray(b'\xe9\x01\x08WL\x02bZ\x00\x01\x00)'))

In [ ]:
import serial
import time

def pump_connect(com_port):
    try:
        pump = serial.Serial(port=com_port, baudrate=9600, parity =serial.PARITY_NONE, bytesize = 8, stopbits = 1,  timeout = None, xonxoff = 0, rtscts = 0)
        print(f"Connected to pump on {com_port}")
        return pump
    except serial.SerialException as e:
        print(f"Error: {e}")
        return None
    
# pump conncetion
com_port_number = 4
pump = pump_connect(f'COM{com_port_number}')

# Run pump 1 clockise at flow rate of 4 mL/min 
Data = bytearray(b'\xE9\x01\x08\x57\x4C\x00\x3D\x09\x00\x01\x01\x26')
pump.write(Data)
time.sleep(5)
# Stop the pump
Data = bytearray(b'\xE9\x01\x08\x57\x4C\x00\x2D\xC6\xC0\x00\x00\x39')
pump.write(Data)
# Close connection
pump.close()